# Evaluation de la qualité des données
Le but de ce notebook est d'identifier et d'analyser les problèmes de qualité des données. 

## Lecture des données

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///data.db', echo=False)
con = engine.connect()
df_patient = pd.read_sql('select * from patient', con=con)
df_pcr = pd.read_sql('select * from test', con=con)
con.close()

### Analyse rapide des données

In [3]:
# quick data check 
df_patient.head()

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
0,221958,matisse,clarke,13.0,rene street,ellenbrook,2527,wa,19710708.0,32.0,08 86018809,westella
1,771155,joshua,elrick,23.0,andrea place,east preston,2074,nsw,19120921.0,34.0,02 97793152,foxdown
2,231932,alice,conboy,35.0,mountain circuit,prospect,2305,nsw,19810905.0,22.0,02 20403934,None
3,465838,sienna,craswell,39.0,cumberlegeicrescent,henty,3620,wa,19840809.0,30.0,02 62832318,jodane
4,359178,joshua,bastiaans,144.0,lowrie street,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None


Le Dataframe **df_patient** contient les informations personnelles sur les patients. Dans le détail, on y trouve les 12 colonnes suivantes qui sont explicites:  
* patient_id : Un  identifiant qui semble représenter un clé	
* given_name	
* surname	
* street_number
* address_1	
* suburb	
* postcode	
* state	
* date_of_birth	
* age	
* phone_number	
* address_2


In [4]:
df_pcr.head()

,patient_id,pcr
0,653091,N
1,347667,Negative
2,708481,Positive
3,148825,Negative
4,150081,Negative


Le Dataframe **df_pcr** contient une colonne patient_id et une colonne pcr qui contient les résultats des tests PCR. On note déjà une première incohérence seulement en regardant les 4 premières lignes de données: les résultats ne sont pas notés de façons consistantes, ainsi **les tests négatifs sont notés N ou Negative**. Nous explorerons ce problème complètement plus tard dans l'analyse. 

## Analyse des données de df_patient

In [5]:
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patient_id     20000 non-null  int64  
 1   given_name     19560 non-null  object 
 2   surname        19575 non-null  object 
 3   street_number  19618 non-null  float64
 4   address_1      19204 non-null  object 
 5   suburb         19788 non-null  object 
 6   postcode       19801 non-null  object 
 7   state          18010 non-null  object 
 8   date_of_birth  17989 non-null  float64
 9   age            16003 non-null  float64
 10  phone_number   19081 non-null  object 
 11  address_2      7893 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 1.8+ MB


Il y a 20 000 lignes dans le Dataframe. Et il y a des données manquantes pour toutes les colonnes excepté la colonne patient_id. On remarque que les colonnes postcode et phone_number sont représenté sous forme de chaine des caractères et non sous forme de nombres. Par contre date_of_birth est représenté comme un nombre décimal, comme l'age. 
La cellule suivante donne le nombre de valeurs nulles pour chaque colonnes. 


In [6]:
df_patient.isnull().sum().sort_values(ascending=False)

address_2        12107
age               3997
date_of_birth     2011
state             1990
phone_number       919
address_1          796
given_name         440
surname            425
street_number      382
suburb             212
postcode           199
patient_id           0
dtype: int64

La colonne avec le plus de données manquantes est address_2. C'est compréhensible car ce n'est qu'un complément d'adresse qui n'est pas toujours nécessaire. Par contre les données manquantes sont problématiques pour les 2 colonnes qui suivent dans la liste: age et date_of_birth. D'ailleurs ces 2 colonnes ont l'air redondantes: On pourrait calculer age à partir de date_of_birth. Mais ce point devra être confirmé par la suite.  
En bas de la liste, on trouve patient_id qui n'a pas de valeurs nulles. 


### Analyse statistique des colonnes numériques

In [7]:
df_patient.describe()

,patient_id,street_number,date_of_birth,age
count,20000.000000,19618.000000,1.798900e+04,16003.000000
mean,553918.938750,98.812468,1.949638e+07,27.534337
std,260234.961111,2481.908686,2.926916e+05,7.807008
min,100005.000000,0.000000,1.290041e+07,0.000000
25%,330194.250000,10.000000,1.924113e+07,24.000000
50%,556327.500000,24.000000,1.950032e+07,29.000000
75%,779419.500000,60.000000,1.975052e+07,33.000000
max,999993.000000,342951.000000,1.999123e+07,92.000000


De la cellule ci-dessus, nous ne pouvons pas trop tirer de conclusion, excepté pour la colonne age. La moyenne pour la colonne age est 27.534337 et la médianne 29 (D'après wikipedia l'age médian pour l'australie est 38 ans). De plus le 75% percentile est 33. Bien que l'age maximal est 92, nous voyons que la majorité des données sont pour des ages autour de 30 ans. Et donc ces données ne sont pas représentative de toute la population. 

## Colonne patient_id
La colonne patient_id semble être une clé. Et de ce fait toutes les valeurs devraient êtres unique pour identifier correctement une personne. Vérifions ce point. 

In [8]:
# Using the option keep=False to show all the duplicated rows. 
# Moreover a row can be duplicated more than once
df_patient[df_patient.duplicated(subset=['patient_id'], keep=False)].sort_values(by=['patient_id'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
12117,109304,zachary,farronato,30.0,outtrim avenue,como,2196,vic,19090801.0,31.0,07 22894061,the reefs
14839,109304,bailey,donaldson,20.0,tardent street,ryde,0812,qld,19580310.0,26.0,07 13479210,None
4386,110207,toby,brock,4.0,merriman crescent,baralaba,3025,nsw,19000424.0,35.0,08 33842007,leitrim
12989,110207,zali,brock,32.0,hedger street,toorak,5038,act,NaN,22.0,08 96818512,None
10184,115791,hannah,clarke,70.0,galmarra street,mayfield,7010,vic,19830828.0,25.0,04 70760611,None
...,...,...,...,...,...,...,...,...,...,...,...,...
10507,990695,erin,braunack,49.0,moondarra street,broken hill,2640,qld,19830122.0,30.0,03 69523317,yuulong
8764,990936,amy,royle,90.0,whittell crescent,coramba,5032,sa,19950326.0,NaN,08 07309295,tewantin plaza
12563,990936,samantha,green,21.0,brierly street,ardrossan,2140,None,19380210.0,29.0,02 51600621,None
2385,994235,trent,stewart-jones,129.0,macfarland crescent,wangaratta,2732,nsw,NaN,NaN,07 98662458,mountview


Ces 403 lignes dupliqués sont vraiment un problème, car on voit que ce sont des personnes différentes. Les duplications ne viennent pas de fautes de frappes dans la saisie du nom, de l'adresse, ...
Regardons le test PCR pour un de ces patient_id dupliqués. 


In [9]:
# Test result for one of the duplicated patient_id 
df_pcr[ df_pcr['patient_id'] == 109304 ]

,patient_id,pcr
1960,109304,N


Le test est négatif pour le patient 109304, mais on n'a aucun élément pour dire s'il s'applique à Mr farronato ou Mr donaldson. Donc la solution sera de supprimer tous les patient_id dupliqués dans le dataframe df_patient. 

### Colonne date_of_birth
La colonne date_of_birth est censé représenter la date de naissance, mais le format utilisé est surprenant: C'est un nombre décimal avec 8 chiffres. Cependant lorsque l'on compare la colonne date_of_birth avec la colonne age, il n'y a aucun lien logique. Les premiers patients trié par date_of_birth sont listés dans la cellule suivante.

In [10]:
df_patient.sort_values(by=['date_of_birth']).head(50)

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
8882,258272,jack,jessup,509.0,stone place,batlow,None,qld,12900410.0,34.0,08 77804659,mayflower retrmnt vlge
11591,986226,lara,white,6.0,collocott crescent,alice springs,5031,None,19000105.0,22.0,04 53214941,bracken leigh
1689,707847,rosa,crook,27.0,wilkins street,cooma,5125,vic,19000106.0,NaN,03 41108108,None
1102,218033,taylah,None,3.0,ern florence crescent,montmorency,5051,vic,19000112.0,NaN,02 83506886,None
11936,726441,timotby,webb,100.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
18880,400767,timothy,webb,10.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
13336,981450,rocca,samantha,29.0,jacka crescent,edgecilff,6024,nsw,19000119.0,38.0,08 42055191,australia arcade
14362,161596,samantha,rocca,29.0,jacka crescent,edgecliff,6024,nsw,19000119.0,38.0,08 42055191,australia arcade
15408,847075,None,haupt,46.0,None,devonport,4871,wa,19000120.0,27.0,03 47375443,None
18101,690620,riley,dixon,47.0,investigator street,hillside,4870,sa,19000121.0,NaN,07 63746321,None


La valeur la plus faible est "12900410.0". On s'attendrai à ce que ce soit le patient le plus agé. Or il n'a que 34 ans.   
Le format de la colonne semble être de la forme:"19xxmmjj" avec mm (1-12) le mois en chiffre, et jj le jour, excepté pour la première ligne. Mais cela semble être une faute de frappe. 
Mais nous ne savons pas à quoi correspond les 4 premiers chiffres "19xx". 
 
Donc telle quelle nous ne savons pas comment utiliser les informations dans la colonne date_of_birth. Nous pouvons essayer d'analyser un peu plus cette colonne. Logiquement les personnes avec le même date_of_birth devrait avoir le même age. 
 

In [11]:
# Duplicated rows for 'date_of_birth', 'age' not null 
df_patient[df_patient.duplicated(subset=['date_of_birth', 'age'], keep=False) & 
           df_patient['date_of_birth'].notnull() & df_patient['age'].notnull()
          ].sort_values(by=['age', 'date_of_birth'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
1609,423412,talixsa,brammy,11.0,wtreet james,maryborough,5333,vic,19060917.0,7.0,02 77001738,tourist park
2407,821864,talissa,brammy,11.0,james street,maryborough,5333,vic,19060917.0,7.0,02 77001738,tourist park
582,876665,madrlixe,beadell,117.0,groom street,wyoning,4218,nsw,19230525.0,7.0,08 81656149,None
3550,549916,madeline,beadell,117.0,groom street,wyoming,4218,nsw,19230525.0,7.0,08 81656149,None
3411,792841,None,eglineton,12.0,helemon street,surry hills,3216,qlg,19300427.0,7.0,02 49550430,north road
...,...,...,...,...,...,...,...,...,...,...,...,...
4971,162433,eva,miles,18.0,collings wtreet,eight mile plains,6232,vic,19941210.0,43.0,07 71242229,rosetta village
12288,169409,eva,miles,91.0,collingsstreet,eight mioe aplains,6232,vic,19941210.0,43.0,07 71242229,rosetta village
17936,753197,eva,miles,91.0,collings street,eight mile plains,6232,vic,19941210.0,43.0,07 71242229,rosetta village
19313,167261,lars,blunden,101.0,armfield place,tarragindi,2031,wa,19080704.0,45.0,04 80689941,None


Mais cela ressemble davantage à des personnes dupliquées. Il serait interresssant de regarder de nouveau ce point, une fois que le dataframe sera nettoyé des doublons.  

### Analyse des doublons
Les colonnes restantes (given_name, surname,  ne surname, street_number, suburb, postcode, state, age, phone_number, et address_2) ne semblent pas poser de problèmes de représentation de données. Sont-elles pour autant fiables? Permettent-elles d'identifier une personne? Y-il-a t'il des doublons dues à des fautes de frappe dans la saisie des information? 
La colonne susceptible de plus précisément identifier une personne est phone_number. 
Regardons s'il y a des lignes dupliquées pour cette colonne. 

In [12]:
df_patient[df_patient.duplicated(subset='phone_number', keep=False)].sort_values(by=['phone_number'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
2634,311830,taaila,None,16.0,mcnicoll place,orange,6230,vic,19180603.0,NaN,02 00325977,None
3872,210155,taalia,None,19.0,mcnicollyplace,orange,6230,vic,19180603.0,64.0,02 00325977,None
16605,525466,taalia,None,19.0,mcnicoll place,orange,6230,vic,19180603.0,26.0,02 00325977,None
248,870746,tyler,able,15.0,marcus clarke street,wannissa,3333,nsw,19830504.0,NaN,02 00493853,None
4986,268616,tyler,able,154.0,marcus clarke street,wanniassa,3333,nsw,19830504.0,NaN,02 00493853,None
...,...,...,...,...,...,...,...,...,...,...,...,...
19899,668176,holly,bristow,35.0,ebden street,dangar island,5024,vic,19501102.0,33.0,None,rocky flat
19901,629087,None,stewart-jones,20.0,jonsson court,aspley,3215,nsw,19490912.0,36.0,None,None
19922,563445,rachel,shepherdson,9.0,calder crescent,junee,6163,None,19151122.0,31.0,None,fernlea
19932,859893,ruby,nascivera,35.0,heard street,roselands,4814,nsw,19841221.0,36.0,None,sheep station


On voit qu'il y 2771 lignes dupliquées. On note que les 3 premières lignes semblent représenter le même patient. Mais il y a des faute de frappe sur address_1 ou street_number. Le bas de la liste montre également que toutes les valeurs nulles sont considérées comme dupliquées, ce qui n'est pas le cas car ces patients qui sont clairement différents. Enlevons les valeurs nulles. 

In [13]:
df_patient[df_patient.duplicated(subset='phone_number', keep=False) &
          df_patient['phone_number'].notnull()].sort_values(by=['phone_number'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
2634,311830,taaila,None,16.0,mcnicoll place,orange,6230,vic,19180603.0,NaN,02 00325977,None
3872,210155,taalia,None,19.0,mcnicollyplace,orange,6230,vic,19180603.0,64.0,02 00325977,None
16605,525466,taalia,None,19.0,mcnicoll place,orange,6230,vic,19180603.0,26.0,02 00325977,None
248,870746,tyler,able,15.0,marcus clarke street,wannissa,3333,nsw,19830504.0,NaN,02 00493853,None
4986,268616,tyler,able,154.0,marcus clarke street,wanniassa,3333,nsw,19830504.0,NaN,02 00493853,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7100,178533,ryab,samakovlis,70.0,ferrier place,kirwan,7301,nsw,NaN,31.0,08 98483679,miami
16577,641889,ryan,samakovlis,70.0,ferrier place,kirwan,7310,nsw,NaN,31.0,08 98483679,miami
5195,912481,kiera,feast,5.0,bateman street,moe,5125,nsw,NaN,33.0,08 99975497,workmans cottage
18207,570917,kierl,feqsa,5.0,bateman street,moe,5125,nsw,NaN,33.0,08 99975497,workmans cottage


Nous avons maintenant 1852 lignes dupliquées. Cet extrait nous montre aussi que les fautes de frappe peuvent apparaître dans n'importe quelle colonne: given_name, surname, ...
Prenons maintenant le cas où il n'y a pas de faute de frappe pour les colonnes 'given_name','surname', 'address_1' et regardons les lignes dupliquées correspondante. 


In [14]:
df_patient[df_patient.duplicated(subset=['given_name','surname', 'address_1'], keep=False) &
          df_patient[['given_name','surname', 'address_1']].notnull().any(axis=1)].sort_values(
          by=['given_name','surname', 'address_1'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
16951,592646,abbey,cadman,5.0,mugga way,burpengary,2554,None,19920502.0,29.0,03 45996223,None
17258,593406,abbey,cadman,5.0,mugga way,burpengary,2545,None,19920502.0,29.0,03 45969923,None
2781,959414,abbey,cropper,17.0,bowden crescent,ringwood,6237,qld,19200312.0,29.0,02 90211615,pangani
5089,560115,abbey,cropper,16.0,bowden crescent,ringwood,6231,qld,19200312.0,21.0,02 90211615,pangani
6627,961858,abbey,green,5.0,wilkins street,ballina,6110,None,19040209.0,7.0,08 75306674,None
...,...,...,...,...,...,...,...,...,...,...,...,...
16552,279300,None,polak,35.0,jess place,annandale,2250,qld,19531130.0,NaN,03 33180608,bonnie doon
1775,310017,None,wehr,57.0,christmas street,mount victoria,3068,nsw,19820227.0,33.0,03 15451529,None
9140,847320,None,wehr,57.0,christmas street,mount victoria,3068,None,19829327.0,33.0,03 15451529,None
7968,215375,None,zimmermann,447.0,birdwood street,beerburrum,4221,vic,19724011.0,29.0,03 10366613,wildefell


L'exemple ci-dessus montre la difficulté de détecter les doublons lorsque l'on ne peut pas s'appuyer sur un idetificateur fort comme phone_number: On a besoin de 3 colonnes ('given_name','surname', 'address_1') pour commencer à identifier un patient. Et l'exemple ci-dessus ne tient pas contre du fait qu'il peut y avoir des fautes de frappe sur ces trois colonnes. 
On voit également qu'il peut y avoir des fautes de frappe sur phone_number.
On note également que la colonne date_of_birth semble être un identificateur de patient, même s'il nous n'avons pas la clé pour récupérer la date de naissance. Donc cette colonne sera utile pour la suppression des doublons. 

In [15]:
df_patient[df_patient.duplicated(subset=['date_of_birth','surname'], keep=False) &
          df_patient[['date_of_birth']].notnull().any(axis=1)].sort_values(
          by=['date_of_birth','surname'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
11936,726441,timotby,webb,100.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
18880,400767,timothy,webb,10.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
561,814998,tahlia,coleman,24.0,furey street,boondall,3163,nsw,19000310.0,24.0,02 02079822,None
7101,340011,tahlia,coleman,24.0,furey street,boondall,3163,nsw,19000310.0,19.0,02 02079822,None
15771,694055,tahlia,coleman,24.0,furey steet,boondall,3163,nsw,19000310.0,19.0,02 32161703,None
...,...,...,...,...,...,...,...,...,...,...,...,...
16782,193878,amy,mason,12.0,balsillie crescent,view bank,2126,vic,19991025.0,12.0,02 48437211,None
3976,810644,juliana,grosvenor,5.0,connelly pace,3023,port noarlunga south,tas,19991215.0,NaN,03 55227740,None
11283,793044,juliana,grosvenor,5.0,connelly place,port noarlunga south,3023,tas,19991215.0,NaN,03 55227740,None
15581,697565,juliana,grosvenor,2.0,connellt place,port noarlunha south,3023,tas,19991215.0,NaN,03 55227740,None


## Analyse des données de df_pcr
Nous allons d'abord vérifier s'il y a des doublons pour la colonne **patient_id**

In [16]:
df_pcr[df_pcr.duplicated(subset=['patient_id'], keep=False)].sort_values(by=['patient_id'])

,patient_id,pcr
169,159500,N
2934,159500,N
8469,169519,Positive
1842,169519,Negative
5618,212217,N
...,...,...
5542,887646,Negative
1025,975512,Negative
2331,975512,N
7477,990695,N


On voit qu'il y a deux cas de figure pour les doublons:  
* Soit la valeur pcr est identique: Il faut dans ce cas juste supprimer le ligne en double 
* Soit les valeur pcr sont différente: Dans ce cas on ne peut pas savoir laquelle est bonne. Il faut donc supprimer les 2 lignes

## Discussion sur le choix de méthode utilisé dans la fonction detect_duplicates
Pour détecter et supprimer les doublons, nous avons d'abord essayé d'utiliser la méthode duplicated des dataframe pandas. 

### Pandas duplicated
Dans la section suivante, nous présentons une tentative d'utilisation de cette méthode et ses limitations.
Nous allons utiliser une succession de dataframe.duplicated appliquées à différentes colonnes de façon heuristique pour éliminer les doublons étapes par étapes. 


In [17]:
def detect_duplicates_pd (df_in):
    df = df_in.copy()
    
    print(df.duplicated(subset=['phone_number']).sum())
    df = df[~(df.duplicated(subset=['phone_number'], keep = 'last') &  
             df[['phone_number']].notnull().any(axis=1))]
    print ("df len :", len(df))

    df = df[~(df.duplicated(subset=['given_name','date_of_birth', 'age'], keep = 'last') &  
             df[['date_of_birth']].notnull().any(axis=1))]
    print ("df len :", len(df))
       
    df = df[~(df.duplicated(subset=['given_name','date_of_birth', 'street_number'], keep = 'last') &  
             df[['date_of_birth']].notnull().any(axis=1))]
    print ("df len :", len(df))
    
    df = df[~(df.duplicated(subset=['given_name','date_of_birth', 'postcode'], keep = 'last') &  
             df[['date_of_birth']].notnull().any(axis=1))]
    print ("df len :", len(df))
    
    df = df[~(df.duplicated(subset=['surname','date_of_birth', 'postcode'], keep = 'last') &  
             df[['date_of_birth']].notnull().any(axis=1))]
    print ("df len :", len(df))
 
    df = df[~(df.duplicated(subset=['surname','date_of_birth', 'street_number'], keep = 'last') &  
             df[['date_of_birth']].notnull().any(axis=1))]
    print ("df len :", len(df))

    df = df[~(df.duplicated(subset=['address_1','date_of_birth','street_number'], keep = 'last') &  
             df[['date_of_birth']].notnull().any(axis=1))]
    print ("df len :", len(df))
    
    df = df[~(df.duplicated(subset=['suburb','age', 'postcode', 'street_number'], keep = 'last') &  
             df[['age']].notnull().any(axis=1))]
    print ("df len :", len(df))
    
    df = df[~(df.duplicated(subset=['suburb','age', 'postcode', 'address_1'], keep = 'last') &  
             df[['age']].notnull().any(axis=1))]
    print ("df len :", len(df))
    print ("Percentage of duplicate: ", 100 * (len(df_in) - len(df))/len(df_in))
    print ("Number of lines removed: ", len(df_in) - len(df))
    print("df out length: ", len(df))
    return df 

In [18]:
df_clean = detect_duplicates_pd(df_patient)

2083
df len : 18835
df len : 18739
df len : 18725
df len : 18722
df len : 18700
df len : 18697
df len : 18687
df len : 18655
df len : 18644
Percentage of duplicate:  6.78
Number of lines removed:  1356
df out length:  18644


Regardons, la qualité du dataframe nettoyé. 

In [19]:
df_clean[df_clean.duplicated(subset=['suburb','age', 'postcode', 'state'], keep = False) & 
          df_clean['age'].notnull()].sort_values(by=
                    ['suburb','age', 'postcode', 'address_1'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
18950,556021,lewis,matthews,NaN,allawah flats,attunga,3216,wa,19690623.0,26.0,08 01674935,None
19494,415387,lewis,matthews,24.0,allawha lats,attunga,3216,wa,19570531.0,26.0,03 95122427,None
11186,299899,evan,reod,2.0,minchin olace,mount stuart,2795,nsw,19000621.0,42.0,03 34884476,None
221,121959,evna,reid,1.0,minchin place,mount stuart,2795,nsw,19160511.0,42.0,03 34844476,None
14615,365161,mikhayla,None,39.0,mckinlay street,st ives,2261,None,19810303.0,29.0,08 07810738,None
15162,103060,rachel,mccarthy,1.0,middleton circuit,st ives,2261,None,19150707.0,29.0,04 46633298,None
3631,939188,thomas,richmond,5.0,blamey crescent,tusmore,6111,vic,NaN,33.0,04 38403905,None
19598,497067,thomas,richjqond,50.0,blamey crsecent,tusmore,6111,vic,19560415.0,33.0,04 38003905,None
8136,651238,stephanie,jolly,13.0,nanson lace,wellard,2486,sa,19540306.0,27.0,02 72656849,None
1699,830546,jolly,stephanie,12.0,nanson place,wellard,2486,sa,19540306.0,27.0,02 72665849,None


Nous voyons qu'il existe encore des lignes dupliquées. Mais il y a 2 lignes qui ne sont pas des doublons: patient_id 365161, 103060. On pourrait essayer et reussir à faire fonctionner cette méthode avec ce dataset. Mais elle pourrait échouer avec un nouveau dataset. Cette méthode ce généralise très mal. Elle a cependant un avantage: elle est rapide à exécuter. 
On peut donc imaginer l'utiliser pour tester la qualité de la fonction detect_duplicates. 

### Bibliothèque FuzzyWuzzy
Le bibliothèque FuzzyWuzzy contient une fonction dedupe qui prend en entré une liste de chaine de caractères et détecte les doublons par 'fuzzy matching'. La méthode pour utiliser cette fonction est de concaténer les colonnes en une chaine de caractères et utiliser dedupe sur cette chaine pour identifier les doublons. 
Mais avant de complètement développer cette méthode, nous avons voulue appliquer dedupe sur une colonne simple: surname. De plus nous allons vérifier la méthode sur un sous-ensemble de 100 lignes. 

In [20]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

In [21]:
dedup = df_patient['surname'][:100].tolist()

# As dedupe method doesn't handle null value, we replace null values by empty string in the list
dedup = [ '' if a is None else a for a in dedup ]
print("length dedup list: ", len(dedup))
dedup = process.dedupe(dedup)
pd_dedup = pd.DataFrame(dedup, columns =['surname'])
print("length pd_dedup after deduplication: ", len(pd_dedup))
pd_dedup.sort_values(by=['surname']).head(20)


length dedup list:  100
length pd_dedup after deduplication:  83


,surname
54,
4,bastiaans
49,bencze
76,binns
68,bligh
11,bristow
63,burford
42,callado
36,callahan
0,clarke


Comparons avec la méthode pandas duplicated. 

In [22]:
df_patient[:100][~df_patient[:100].duplicated(subset=['surname'], keep='last')].sort_values(by=['surname']).head(20)

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
4,359178,joshua,bastiaans,144.0,lowrie street,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None
61,339629,jacqueline,bencze,38.0,andrews street,somers,3126,qld,19330804.0,NaN,08 01253632,birrabilla
92,323165,taylor,binns,2.0,dunbar street,mount annan,4455,wa,19000509.0,11.0,08 99185004,sec 528
81,559773,aurora,bligh,2.0,batman street,clayfield,4745,nsw,NaN,NaN,07 40832133,roachdale
12,999159,joshua,bristow,21.0,thake place,oxley vale,7004,nsw,19650125.0,33.0,08 93848698,None
76,704365,sienna,burford,7.0,hampton circuit,winston hills,3191,nsw,NaN,20.0,None,None
49,655839,teileah,callado,49.0,darambal street,dulwich hill,6018,vic,19280717.0,20.0,02 56099733,None
41,748143,matthew,callahan,46.0,garrad court,narrandera,3088,vic,19970807.0,31.0,04 74566827,None
91,661353,beth,clarke,50.0,osmand nstreet,port macquarie,0836,qld,19750820.0,NaN,08 91194518,None
17,429365,None,coffey,1409.0,musson close,beaconsfield upper,5019,wa,NaN,30.0,03 13369308,silverweir


Nous voyons que la méthode FuzzyWuzzy dedupe semble fonctionner. Cependant il y a quelques erreurs comme craswell qui est pris comme doublon de crosswell. Ce type de problème pourrait se réduire en fusionnant toute les colonnes. Cependant le problème le plus important est la performance. En effet avec 100 éléments, le traitement prends quelques secondes. Mais à partir de 5000 éléments le temps de traitement explose. Donc il n'est pas envisageable d'utiliser cette méthode avec 20 000 éléments. 

### Bibliothèque recordlinkage
Cette bibliothèque peut paraitre plus difficile à utiliser au début mais elle a plusieurs avantages. La [documentation](https://recordlinkage.readthedocs.io/en/latest/about.html) est très bien faite et cette bibliothèque manipule des dataframe pandas nativement. Cette bibliothèque contient une méthode permettant d'identifier des doublons dans un dataframe. 
Le fonctionnement est résumé ci-après: 
* Contruire un index: C'est une liste de paire de lignes qui peuvent être potentiellement des doublons. Cette index peut être contruit avec une ou plisieurs colonnes 
* Ajouter la liste de colonnes qui doivent être comparées 
* Comparer les lignes du dataframe en se basant sur les colonnes et l'index défini au préalable 
* Le résultat de la comparaison est un dataframe avec 2 index et un score pour chaque colonnes 

Regardons ci-desous comment fonctionne cette bibliothèque.   

In [23]:
import recordlinkage

In [24]:
# Indexation step
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='address_1', window=3)

dupe_candidate_links = dupe_indexer.index(df_patient)

# Comparison step
compare_dupes = recordlinkage.Compare()

# Columns to compore 
compare_dupes.string('given_name', 'given_name', method='jarowinkler', threshold=0.85, label='given_name')
compare_dupes.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_dupes.exact('street_number', 'street_number', label='street_number')
compare_dupes.string('address_1', 'address_1', threshold=0.85, label='address_1')
compare_dupes.string('suburb', 'suburb', threshold=0.85, label='suburb')
compare_dupes.exact('postcode', 'postcode', label='postcode')
compare_dupes.exact('state', 'state', label='state')
compare_dupes.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_dupes.exact('age', 'age', label='age')
compare_dupes.string('phone_number', 'phone_number', threshold=0.85, label='phone_number')
compare_dupes.exact('address_2', 'address_2', label='address_2')

# dupes_features: Dataframe with new index and score
dupes_features = compare_dupes.compute(dupe_candidate_links, df_patient)



In [25]:
dupes_features

given_name  surname  street_number  address_1  suburb  postcode  \
779   427           0.0      0.0              0        0.0     0.0         0   
14010 427           0.0      0.0              0        0.0     0.0         0   
      12820         0.0      0.0              0        0.0     0.0         0   
      13051         0.0      0.0              0        0.0     0.0         0   
17644 11430         0.0      0.0              0        0.0     0.0         0   
...                 ...      ...            ...        ...     ...       ...   
19999 10806         0.0      0.0              0        0.0     0.0         0   
      10926         0.0      0.0              0        0.0     0.0         0   
      12722         0.0      0.0              0        0.0     0.0         0   
      14594         0.0      0.0              0        0.0     0.0         0   
      18754         0.0      0.0              0        0.0     0.0         0   

             state  date_of_birth  age  phone_number  address_2  
779   427        1              0    0           0.0          0  
14010 427        0              0    0           0.0          0  
      12820      0              0    0           0.0          0  
      13051      0              0    0           0.0          0  
17644 11430      0              0    0           0.0          0  
...            ...            ...  ...           ...        ...  
19999 10806      1              0    1           0.0          0  
      10926      0              0    0           0.0          0  
      12722      1              0    1           0.0          0  
      14594      0              0    0           0.0          0  
      18754      0              0    0           0.0          0  

[170245 rows x 11 columns]

Le cellule ci-dessus montre la structure du dataframe dupes_features. La taille de l'index de recordlinkage construit est 170245 (le nombre de lignes du dataframe). On voit également l'index hierarchique du dataframe représentant des paires de lignes. On peut également voir les 11 colonnes comparées et les scores. On voit par exemple que pour la paire de lignes (19999, 10806) les colonnes state et age sont identiques. Regardons la répartition de la somme des scores par lignes.  


In [26]:
dupes_features.sum(axis=1).value_counts().sort_index(ascending=False)

11.0       26
10.0      255
9.0       555
8.0       577
7.0       345
6.0       114
5.0        24
4.0        17
3.0      1004
2.0     20884
1.0     87271
0.0     59173
dtype: int64

Toutes les lignes qui ont un score supérieur ou égal à 5 sont probablement des doublons. 

In [27]:
potential_dupes = dupes_features[dupes_features.sum(axis=1) > 4].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'given_name':'address_2'].sum(axis=1)
potential_dupes

,level_0,level_1,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2,Score
0,10533,8595,1.0,1.0,0,1.0,1.0,1,1,1,0,1.0,0,8.0
1,12857,8595,1.0,1.0,0,1.0,1.0,1,1,1,1,1.0,0,9.0
2,13280,8595,1.0,1.0,0,1.0,1.0,0,1,1,1,1.0,0,8.0
3,14399,13793,1.0,1.0,1,1.0,1.0,1,0,1,0,1.0,0,8.0
4,17159,13793,1.0,1.0,0,1.0,1.0,1,0,1,0,1.0,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,19598,8304,1.0,1.0,1,1.0,0.0,1,1,1,1,0.0,0,8.0
1892,19598,10214,1.0,1.0,1,1.0,1.0,1,1,1,1,1.0,0,10.0
1893,19944,742,1.0,1.0,1,1.0,1.0,1,1,1,1,1.0,0,10.0
1894,19944,2737,1.0,1.0,0,1.0,1.0,0,1,0,1,1.0,0,7.0


Vérifions ce point avec un exemple. 

In [28]:
potential_dupes[potential_dupes['Score'] == 5.0].head()

,level_0,level_1,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2,Score
48,5214,448,0.0,0.0,1,0.0,1.0,0,1,0,1,1.0,0,5.0
80,19952,9725,1.0,1.0,0,1.0,0.0,1,0,0,0,1.0,0,5.0
172,12588,5654,0.0,1.0,0,1.0,1.0,1,0,1,0,0.0,0,5.0
173,12588,9140,0.0,1.0,0,1.0,1.0,1,0,0,1,0.0,0,5.0
305,13055,1136,0.0,0.0,1,1.0,1.0,1,0,0,0,1.0,0,5.0


In [29]:
df_patient.loc[5214,:]

patient_id              351671
given_name             riddell
surname                caitlin
street_number               14
address_1        carina street
suburb               st albans
postcode                  5095
state                      qld
date_of_birth              NaN
age                          9
phone_number       08 62513595
address_2                 None
Name: 5214, dtype: object

In [30]:
df_patient.loc[448,:]

patient_id              156958
given_name             caitlin
surname                riddell
street_number               14
address_1        carinassgreet
suburb               st albans
postcode                  5059
state                      qld
date_of_birth              NaN
age                          9
phone_number       08 62513595
address_2                 None
Name: 448, dtype: object

Bien que given_name et surname soient inversées pour cette personne, et qu'il y ai un certain nombre d'erreur du à des faute de frappe, nous pouvons conclure que c'est la même personne.  
   
Pour supprimer les doublons il suffit de supprimer de df_patient toutes les lignes de la colonne level_1 du dataframe potential_dupes.


In [31]:
df_clean = df_patient.drop(axis=0, index=potential_dupes['level_1'])
df_clean

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
0,221958,matisse,clarke,13.0,rene street,ellenbrook,2527,wa,19710708.0,32.0,08 86018809,westella
1,771155,joshua,elrick,23.0,andrea place,east preston,2074,nsw,19120921.0,34.0,02 97793152,foxdown
2,231932,alice,conboy,35.0,mountain circuit,prospect,2305,nsw,19810905.0,22.0,02 20403934,None
3,465838,sienna,craswell,39.0,cumberlegeicrescent,henty,3620,wa,19840809.0,30.0,02 62832318,jodane
4,359178,joshua,bastiaans,144.0,lowrie street,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,368617,abby,None,19.0,john cleland crescent,boyne island,2290,nsw,19570210.0,NaN,02 92489054,glenmore
19996,796658,dale,priest,19.0,bellchambers crescent,hoppers crossing,3033,wa,19590619.0,31.0,07 20549476,None
19997,511885,alessia,mawlai,4.0,miller street,smithfield,3616,tas,NaN,26.0,03 83466737,None
19998,985932,hannah,mason,1.0,halligan place,beaumaris,2443,wa,19140304.0,25.0,04 41125514,None


Vérifions la qualité de la suppression des doublons. 

In [32]:
df_clean[df_clean.duplicated(subset=['given_name','surname', 'phone_number'], keep = False) & 
          df_clean[['given_name','surname', 'phone_number']].notnull().any(axis=1)].sort_values(by=
                    ['given_name','surname', 'phone_number'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
5498,524032,abbey,cadman,5.0,mugga ceay,burpengary,2545,None,19920592.0,29.0,03 45969923,None
17258,593406,abbey,cadman,5.0,mugga way,burpengary,2545,None,19920502.0,29.0,03 45969923,None
11353,612447,abby,geraghty,48.0,place bird,macquarie fields,2210,nsw,19470921.0,26.0,07 31638236,riverbend
16959,200647,abby,geraghty,48.0,bird place,macquarie fields,2210,nsw,19470921.0,27.0,07 31638236,riverbend
1339,402775,adam,kempe,183.0,None,march,5070,vic,19240317.0,32.0,04 56431962,meath mews retrmnt vlge
...,...,...,...,...,...,...,...,...,...,...,...,...
11705,993844,None,fitzpatrick,4.0,None,russell lea,2340,nsw,19022000.0,30.0,04 17727347,swannie
1100,662926,None,green,93.0,doolette place,alice springs,2444,nsw,19330103.0,31.0,None,lonesome dove
5694,209162,None,green,NaN,glass place,fish creek,3802,None,19790110.0,12.0,None,None
602,472370,None,ryan,4.0,lucas place,kurmond,4216,qld,NaN,NaN,None,None


Nous voyons qu'il existe encore des doublons: les 2 premières lignes par exemple. Pourtant les valeurs de plus de 5 colonnes sont égales: given_name, surname, street_number, age, suburb, ...
Vérifions si ces lignes sont dans l'index de recordlinkage. 

In [33]:
dupes_features.loc[5498,:]

,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
814,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
1665,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
2137,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
3395,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
4325,0.0,0.0,0,0.0,0.0,0,0,0,1,0.0,0


In [34]:
dupes_features.loc[17258,:]

,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
116,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
4547,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
16708,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0
65,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0
1093,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0
1113,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0
1667,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0
3254,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0
4127,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0
4848,0.0,0.0,0,1.0,0.0,0,0,0,0,0.0,0


Nous voyons qu'il n(y a pas de paire (5498, 17258) dans l'index recordlinkage. Donc il n'est pas possible de détecter par la suite si ces 2 lignes sont dupliquées. Nous devons donc augmenter la taille de l'index recordlinkage pour améliorer la qualité de la deduplication. C'est ce que nous avons fait dans la version finale de la fonction detect_duplicates. 
Nous avons donc choisie la solution basée sur la bibliothèque recordlinkage pour implémenter la fonction detect_duplicates pour les raisons suivantes: 
* Solution plus performante: le traitement prends quelques dizaines de secondes pour 20 000 lignes 
* La qualité des résultats est optimale 
* Cette solution peut être généralisée et appliquée à d'autres dataset sans trop d'effort
